## Sample Python Code for Plato Renovation Inc.
- Part A of this code illustrates how to estimate the historical collection rates across different types of accounts using data from 2010 to 2019. Part B of this codes calculates the total receivables across different types of accounts for the transactions in Q1 2020.
- Before running the code, you need to download two datasets (i.e., *Plato_Dataset_10_19.csv* and *Plato_Dataset_2020Q1.csv*) and save them in your local folder.
- **Your task** is to understand the coding procedures and then pick the right numbers from the outputs to fill in the blanks for following analyses in the doc file.
- Tip: Learning from existing programs is a very important skill for programmers. What's even more important is to be able to adapt those programs for your own particular use. If you want to sharpen your programming skills, you can try to write your own program to address the case questions after understanding this one. 

### Part A: Estimating detailed collection rates usng historical transaction data 
Dataset: Plato_Dataset_10_19.csv

---
**Import the dataset**
- The new datetime module supplies classes for manipulating dates and times.

In [4]:
import datetime as dt
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_csv('Plato_Dataset_10_19.csv', parse_dates = ['bus_date', 'payment_date', 'qend'])

---
**Keep transactions with outstanding accounts receivables by each quarter end**

<font color='red'>***Question 1:***</font> *Which transactions are kept in the new dataset? Why do those transactiosn represent outstanding accounts receivables? You might have to search online the isnull( ) function.*

Your answer: The transactions that are kept are ones without a due date (null) and the ones with a due date after the quarter's end. These are included because the ones at the quarter's end are supposed to be collected as such while the others will likely still be outstanding after this quarter.

In [6]:
df = df[(df['payment_date'].isnull()) | (df['payment_date'] > df['qend'])]
df

,bus_date,district,sales,upfront,payment_date,qend
15,2010-01-05,Tourism,13920,2700,NaT,2010-03-31
18,2010-01-05,Residence,8200,1600,NaT,2010-03-31
20,2010-01-06,Tourism,8640,1700,2010-04-01,2010-03-31
23,2010-01-07,Residence,10000,2000,NaT,2010-03-31
24,2010-01-07,Residence,11000,2200,2010-04-16,2010-03-31
...,...,...,...,...,...,...
20643,2019-12-31,Business,10140,2000,2020-01-26,2019-12-31
20644,2019-12-31,Tourism,13459,2600,2020-01-14,2019-12-31
20645,2019-12-31,Residence,4160,800,2020-02-09,2019-12-31
20646,2019-12-31,Residence,12480,2400,2020-01-11,2019-12-31


---
**Create necessary variables**

In [14]:
df['age'] = (df['qend'] - df['bus_date']).dt.days + 1

<font color='red'>***Question 2:***</font> *Can you explain what the below code does? You might have to search online the pandas.cut() function.*

Your answer: This is finding out how many days have elapsed since you paid and there is one day added in case someone pays the same day they are billed.

In [15]:
#The upper limit of age is 92. For simplicity, we still assign these transactions into (60, 90)
df['due_age'] = pd.cut(df['age'], bins=[0, 30, 60, 92], labels=['(0, 30)', '(30, 60)', '(60, 90)'])

In [16]:
df['year'] = pd.DatetimeIndex(df['bus_date']).year

In [17]:
df['quarter'] = pd.DatetimeIndex(df['bus_date']).quarter

<font color='red'>***Question 3:***</font> *Can you explain what the below code does? You might have to search online both isin( ) and astype( ) function. What does the new variable represent?*

Your answer: This is trying to find out if they were operating in a year that had the bridge close. This new variable represetns  abinary condition with 0 being it was not a birdge year and 1 being a birdge year.

In [18]:
df['bridge_year'] = df['year'].isin([2010,2015]).astype(int)

<font color='red'>***Question 4:***</font> *Can you explain what the below code does? What does the new variable represent?*

Your answer: This is chekcing to see if payment is received. If it was the payment date would return a 0 becuase it would not be null and the 1 - 0 would return a 1 or TRUE meaning a payment has been received.

In [19]:
df['payment_received'] = 1 - df['payment_date'].isnull().astype(int)

---
**Estimate collection rates across different dimensions**

<font color='red'>***Question 5:***</font> *Can you explain what the below code does? You might have to search online about the use of groupby( ) function.*

Your answer: Its making groups based on the bins and different values we have created and matches the values we are looking for by filtering them. For the first we have the payment received % filtered by the age bins we have. You can also have multiple subgroups like on the last line of this section splitting up each of them.

In [20]:
df.groupby('due_age')['payment_received'].mean()

due_age
(0, 30)    0.942
(30, 60)   0.881
(60, 90)   0.671
Name: payment_received, dtype: float64

In [21]:
df.groupby('district')['payment_received'].mean()

district
Business    0.913
Residence   0.890
Tourism     0.880
Name: payment_received, dtype: float64

In [22]:
df.groupby('bridge_year')['payment_received'].mean()

bridge_year
0   0.912
1   0.833
Name: payment_received, dtype: float64

In [23]:
df.groupby('quarter')['payment_received'].mean()

quarter
1   0.895
2   0.901
3   0.892
4   0.891
Name: payment_received, dtype: float64

In [24]:
df.groupby(['bridge_year','due_age', 'district'])['payment_received'].mean()

bridge_year  due_age   district 
0            (0, 30)   Business    0.953
                       Residence   0.945
                       Tourism     0.962
             (30, 60)  Business    0.895
                       Residence   0.880
                       Tourism     0.931
             (60, 90)  Business    0.716
                       Residence   0.623
                       Tourism     0.742
1            (0, 30)   Business    0.957
                       Residence   0.912
                       Tourism     0.826
             (30, 60)  Business    0.899
                       Residence   0.876
                       Tourism     0.712
             (60, 90)  Business    0.794
                       Residence   0.735
                       Tourism     0.493
Name: payment_received, dtype: float64

---
### Part B: Calculating total accounts receivables for 2020 Q1
Dataset: Dataset_2020Q1 for Data Analytics.csv

In [7]:
df20 = pd.read_csv('Plato_Dataset_2020Q1.csv', parse_dates = ['bus_date', 'payment_date', 'qend'])

In [8]:
df20 = df20[(df20['payment_date'].isnull()) | (df20['payment_date'] > df20['qend'])]

In [9]:
df20['receivables'] = df20['sales'] - df20['upfront']

In [10]:
df20['age'] = (df20['qend'] - df20['bus_date']).dt.days + 1

In [11]:
df20['due_age'] = pd.cut(df20['age'], bins=[0, 30, 60, 92], labels=['(0, 30)', '(30, 60)', '(60, 90)'])

**Calculate total accounts receivables across different dimensions**

In [12]:
df20['receivables'].sum()

1796501

In [13]:
df20.groupby(['district'])['receivables'].sum()

district
Business     394350
Residence    630954
Tourism      771197
Name: receivables, dtype: int64

In [14]:
df20.groupby(['due_age'])['receivables'].sum()

due_age
(0, 30)     1053488
(30, 60)     505485
(60, 90)     237528
Name: receivables, dtype: int64

In [17]:
df20.groupby(['due_age', 'district'])['receivables'].sum()

due_age   district 
(0, 30)   Business     203729
          Residence    361637
          Tourism      488122
(30, 60)  Business     144569
          Residence    226102
          Tourism      134814
(60, 90)  Business      46052
          Residence     43215
          Tourism      148261
Name: receivables, dtype: int64

**Calculating the total actual non-paid accounts (assuming we know this information by Q2 2020)**

In [18]:
df20[df20['payment_date'].isnull()]['receivables'].sum()

383527